## This notebook shows how to use Orchestrator APIs for user experiments

In [1]:
import os
from fabric_cli.cli.credential import CredMgr
from fabric_cli.cli.orchestrator import Orchestrator
import json

### Fabric Tokens
Fabric has 2 kinds of tokens:<br>
- Identity : required for Control/Measurement Framework APIs. Identity Token is valid upto an hour.
- Refresh : required to generate new Identity Tokens valid. Refresh Token is valid for 24 hours.

Fabric Identity token is required for Control/Measurement Framework APIs.<br>

When user logins to Jupyterhub after authenticating against CILogon, OIDC refresh token is derived. <br>
This token is available as the environment variable `CILOGON_REFRESH_TOKEN`.<br>

On the first login, we use `CILOGON_REFRESH_TOKEN` to generate new Fabric Identity Token and Fabric Refresh Token.<br>
For any subsequent use, we use Fabric Refresh Token. On every refresh, Fabric Refresh Token is changed and updated.<br>

NOTE: These steps are required for any experiments on Fabric Testbed.<br>

In [2]:
not_found=False
fabric_refresh_token=None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

Fabric Refresh Token https://cilogon.org/oauth2/refreshToken/54a4f3e9e2314f682b9e88b3df4ed47c/1609861320762
CILOGON_REFRESH_TOKEN environment variable: https://cilogon.org/oauth2/refreshToken/2e7db1c1abdae5c8f64d9b1893accbb/1609861275809


### Get new Fabric Identity Token and update Fabric Refresh Token

In [3]:
try:
    refresh_res = CredMgr.refresh_token(project_name='all', scope='all', refresh_token=fabric_refresh_token)
    print("New Tokens received: {}".format(json.dumps(refresh_res)))
    fabric_id_token=refresh_res['id_token']
    fabric_refresh_token=refresh_res['refresh_token']
    print()
    print("New Refresh Token: {}".format(fabric_refresh_token))
    print()
    print("Stored new Refresh Token")
    %store fabric_refresh_token
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

New Tokens received: {"id_token": "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImI0MTUxNjcyMTExOTFlMmUwNWIyMmI1NGIxZDNiNzY2N2U3NjRhNzQ3NzIyMTg1ZTcyMmU1MmUxNDZmZTQzYWEifQ.eyJlbWFpbCI6Imt0aGFyZTEwQGVtYWlsLnVuYy5lZHUiLCJnaXZlbl9uYW1lIjoiS29tYWwiLCJmYW1pbHlfbmFtZSI6IlRoYXJlamEiLCJuYW1lIjoiS29tYWwgVGhhcmVqYSIsImlzcyI6Imh0dHBzOi8vY2lsb2dvbi5vcmciLCJzdWIiOiJodHRwOi8vY2lsb2dvbi5vcmcvc2VydmVyQS91c2Vycy8xMTkwNDEwMSIsImF1ZCI6ImNpbG9nb246L2NsaWVudF9pZC8yZjQ1NGU4YzUzMWJhMDNkOTczNGZhNjg3YzMyYTE5IiwidG9rZW5faWQiOiJodHRwczovL2NpbG9nb24ub3JnL29hdXRoMi9pZFRva2VuLzQ0YzNlZjQ4OGQ1MGY0NWRhODQwNTY0MjUxZDRjMWYzLzE2MDk4NjEyNzU1OTUiLCJhdXRoX3RpbWUiOiIxNjA5ODYxMjc1IiwiZXhwIjoxNjA5ODY0OTQ4LCJpYXQiOjE2MDk4NjEzNDgsInByb2plY3RzIjp7fSwicm9sZXMiOlsicHJvamVjdC1sZWFkcyIsImFiZjAwMTRlLTcyZjUtNDRhYi1hYzYzLTVlYzVhNWRlYmJiOC1wbyIsImFiZjAwMTRlLTcyZjUtNDRhYi1hYzYzLTVlYzVhNWRlYmJiOC1wbSJdLCJzY29wZSI6ImFsbCJ9.QYe4wZX0J5k-QUPyg7GspCC76am2bjLecaXE7JVGNcYcO0xtDc9WfhFOkB76s5RBnBh7pL3cPuRDmaSdZkIgYV6wPoAMJdkL1F0SZfgoQOD0KI6Fyk7baFarS

### Orchestrator API example to query for available resources

In [4]:
resources = Orchestrator.resources(id_token=fabric_id_token)

print(resources)

{'value': '<?xml version="1.0" encoding="UTF-8"?><graphml xmlns="http://graphml.graphdrawing.org/xmlns" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://graphml.graphdrawing.org/xmlns http://graphml.graphdrawing.org/xmlns/1.0/graphml.xsd"><key id="GraphID" for="node" attr.name="GraphID" attr.type="string"/><key id="labels" for="node" attr.name="labels" attr.type="string"/>\n<key id="Labels" for="node" attr.name="Labels" attr.type="string"/><key id="label" for="node" attr.name="label" attr.type="string"/><key id="Capacities" for="node" attr.name="Capacities" attr.type="string"/><key id="Layer" for="node" attr.name="Layer" attr.type="string"/><key id="labels" for="node" attr.name="labels" attr.type="string"/><key id="Name" for="node" attr.name="Name" attr.type="string"/><key id="LabelDelegations" for="node" attr.name="LabelDelegations" attr.type="string"/><key id="Type" for="node" attr.name="Type" attr.type="string"/><key id="ADMGraphIDs" for="node" attr.n